In [99]:
import os
import json
import pandas as pd
import traceback
from langchain_openai import AzureOpenAI
from langchain.chat_models import azure_openai
from dotenv import load_dotenv

In [100]:
load_dotenv()

True

In [101]:
DEPLOYMENT_NAME =os.getenv("deployment_name")
API_KEY=os.getenv("api_key")
API_VERSION=os.getenv("api_version")
ENDPOINT=os.getenv("azure_endpoint")

In [102]:
#print(API_KEY)

In [103]:
llm = AzureOpenAI(
    deployment_name=DEPLOYMENT_NAME,
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT,
    temperature=0.5,
    max_tokens=600
)

In [104]:
llm

AzureOpenAI(client=<openai.resources.completions.Completions object at 0x000001C96E95FD30>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001C96FFB9A80>, temperature=0.5, max_tokens=600, openai_api_key='1ebda9eb5627499591c8b3521f94e5eb', openai_proxy='', azure_endpoint='https://dp0303014.openai.azure.com/', deployment_name='dp140303', openai_api_version='2023-12-01-preview', openai_api_type='azure')

In [105]:
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [106]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [107]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [108]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [109]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [110]:
quiz_chain= LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [111]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [112]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [113]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [114]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [115]:
file_path=r"C:\Users\DebasisPanigrahy\Desktop\My_Work\GenAi\MCQ_Gen\data.txt"

In [116]:
file_path

'C:\\Users\\DebasisPanigrahy\\Desktop\\My_Work\\GenAi\\MCQ_Gen\\data.txt'

In [117]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [118]:
TEXT

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contrib

In [119]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [120]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [121]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Machine Learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:


{"1": {"mcq": "Who coined the term 'machine learning'?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "a"}, "2": {"mcq": "What was the purpose of the 'learning machine' developed by Raytheon Company in the early 1960s?", "options": {"a": "To analyze sonar signals", "b": "To classify data", "c": "To make predictions for future outcomes", "d": "To recognize patterns"}, "correct"

In [122]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2502
Prompt Tokens:2095
Completion Tokens:407
Total Cost:0.0039565


In [123]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [124]:
quiz_str=response.get("quiz")

In [125]:
quiz_str

'\n\n{"1": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "a"}, "2": {"mcq": "What was the purpose of the \'learning machine\' developed by Raytheon Company in the early 1960s?", "options": {"a": "To analyze sonar signals", "b": "To classify data", "c": "To make predictions for future outcomes", "d": "To recognize patterns"}, "correct": "a"}, "3": {"mcq": "What did Alan Turing propose in his paper \'Computing Machinery and Intelligence\'?", "options": {"a": "Can machines think?", "b": "Can machines do what we can do?", "c": "Can machines learn from experience?", "d": "Can machines understand human language?"}, "correct": "b"}, "4": {"mcq": "What is the main goal of modern-day machine learning?", "options": {"a": "To classify data", "b": "To make predictions for future outcomes", "c": "To study human cognitive processes", "d": "To understand human language"}, "correct": 

In [126]:
quiz_str

'\n\n{"1": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "a"}, "2": {"mcq": "What was the purpose of the \'learning machine\' developed by Raytheon Company in the early 1960s?", "options": {"a": "To analyze sonar signals", "b": "To classify data", "c": "To make predictions for future outcomes", "d": "To recognize patterns"}, "correct": "a"}, "3": {"mcq": "What did Alan Turing propose in his paper \'Computing Machinery and Intelligence\'?", "options": {"a": "Can machines think?", "b": "Can machines do what we can do?", "c": "Can machines learn from experience?", "d": "Can machines understand human language?"}, "correct": "b"}, "4": {"mcq": "What is the main goal of modern-day machine learning?", "options": {"a": "To classify data", "b": "To make predictions for future outcomes", "c": "To study human cognitive processes", "d": "To understand human language"}, "correct": 

In [127]:
quiz_str=eval('quiz_str')

In [128]:
type(quiz_str)

str

In [129]:
quiz_dict=json.loads(quiz_str)

In [130]:

quiz_dict

{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Warren McCulloch'},
  'correct': 'a'},
 '2': {'mcq': "What was the purpose of the 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'To analyze sonar signals',
   'b': 'To classify data',
   'c': 'To make predictions for future outcomes',
   'd': 'To recognize patterns'},
  'correct': 'a'},
 '3': {'mcq': "What did Alan Turing propose in his paper 'Computing Machinery and Intelligence'?",
  'options': {'a': 'Can machines think?',
   'b': 'Can machines do what we can do?',
   'c': 'Can machines learn from experience?',
   'd': 'Can machines understand human language?'},
  'correct': 'b'},
 '4': {'mcq': 'What is the main goal of modern-day machine learning?',
  'options': {'a': 'To classify data',
   'b': 'To make predictions for future outcomes',
   'c': 'To study human cognitive processes',
   'd': 'T

In [131]:
len(quiz_dict)

5

In [132]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [133]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'a'},
 {'MCQ': "What was the purpose of the 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: To analyze sonar signals | b: To classify data | c: To make predictions for future outcomes | d: To recognize patterns',
  'Correct': 'a'},
 {'MCQ': "What did Alan Turing propose in his paper 'Computing Machinery and Intelligence'?",
  'Choices': 'a: Can machines think? | b: Can machines do what we can do? | c: Can machines learn from experience? | d: Can machines understand human language?',
  'Correct': 'b'},
 {'MCQ': 'What is the main goal of modern-day machine learning?',
  'Choices': 'a: To classify data | b: To make predictions for future outcomes | c: To study human cognitive processes | d: To understand human language',
  'Correct': 'b'},
 {'MCQ': 'What is the main difference between machine lear

In [134]:
df=pd.DataFrame(quiz_table_data)

In [135]:
df

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a: Arthur Samuel | b: Donald Hebb | c: Walter ...,a
1,What was the purpose of the 'learning machine'...,a: To analyze sonar signals | b: To classify d...,a
2,What did Alan Turing propose in his paper 'Com...,a: Can machines think? | b: Can machines do wh...,b
3,What is the main goal of modern-day machine le...,a: To classify data | b: To make predictions f...,b
4,What is the main difference between machine le...,a: Data mining focuses on the discovery of unk...,a


In [136]:
df.to_csv("Data_Science_Quiz.csv",index=False)